<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
#!pip install glob
#!pip install pandas
#!pip install requests
#!pip install datetime

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime
import requests

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2021-12-26 18:17:34--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2021-12-26 18:17:34 (75.1 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2021-12-26 18:17:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [5]:
columns=['Name','Market Cap (US$ Billion)']

In [16]:
def extract():
    df = pd.DataFrame(columns=columns)
    for file_ in glob.glob("*.json"):
        df = df.append(extract_from_json(file_))
    return df

                                Name  Market Cap (US$ Billion)
35   Banco Bilbao Vizcaya Argentaria                    60.678
36                   Japan Post Bank                    60.525
37       The Bank of New York Mellon                    59.823
38  Shanghai Pudong Development Bank                    58.796
39           Industrial Bank (China)                    57.388
105
105
                               Name  Market Cap (US$ Billion)
0   Banco Bilbao Vizcaya Argentaria                    60.678
1                   Japan Post Bank                    60.525
2       The Bank of New York Mellon                    59.823
3  Shanghai Pudong Development Bank                    58.796
4           Industrial Bank (China)                    57.388


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [19]:
df_bank_market = extract()
# set the parameter index_col to 0
df_bank_market = df_bank_market.reset_index(drop=True)
df_bank_market.head()

,Name,Market Cap (US$ Billion)
0,Banco Bilbao Vizcaya Argentaria,60.678
1,Japan Post Bank,60.525
2,The Bank of New York Mellon,59.823
3,Shanghai Pudong Development Bank,58.796
4,Industrial Bank (China),57.388


In [44]:
# Write your code here
df_exc_rates = pd.read_csv('exchange_rates.csv')
# Rename column
df_exc_rates.rename(columns={'Unnamed: 0': 0}, inplace=True)
print(df_exc_rates.head())
exchange_rate = float(df_exc_rates.loc[df_exc_rates[0] == 'GBP', 'Rates'])
print(exchange_rate)

     0     Rates
0  AUD  1.297088
1  BGN  1.608653
2  BRL  5.409196
3  CAD  1.271426
4  CHF  0.886083
0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [35]:
def transform(data_to_transform: pd.DataFrame):
    # 1. Changes the Market Cap (US$ Billion) column from USD to GBP
    data_to_transform['Market Cap (US$ Billion)'] = data_to_transform['Market Cap (US$ Billion)'].apply(lambda x: x*exchange_rate)
    # 2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
    data_to_transform['Market Cap (US$ Billion)'] = data_to_transform['Market Cap (US$ Billion)'].apply(lambda x: round(x, 3))
    # 3. Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)
    data_to_transform.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=True)
    return data_to_transform

df = transform(df_bank_market)
df.head()

,Name,Market Cap (GBP$ Billion)
0,Banco Bilbao Vizcaya Argentaria,44.440
1,Japan Post Bank,44.328
2,The Bank of New York Mellon,43.814
3,Shanghai Pudong Development Bank,43.062
4,Industrial Bank (China),42.031


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [49]:
def load(data_to_load, file_name: str = 'bank_market_cap_gbp.csv'):
    data_to_load.to_csv(file_name, index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [40]:
def log(msg):
    msg = f"{datetime.now().replace(microsecond=0)}: {msg}"
    print(msg)
    # log to file
    with open('log_file.txt', 'a') as file_:
        file_.write(msg)

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [41]:
# Write your code here
log("ETL Job Started")
log("Extract phase Started")

2021-12-26 19:25:39: ETL Job Started
2021-12-26 19:25:39: Extract phase Started


### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [42]:
# Call the function here
df_bank_market = extract()
# set the parameter index_col to 0
df_bank_market = df_bank_market.reset_index(drop=True)
# Print the rows here
df_bank_market.head()

,Name,Market Cap (US$ Billion)
0,Banco Bilbao Vizcaya Argentaria,60.678
1,Japan Post Bank,60.525
2,The Bank of New York Mellon,59.823
3,Shanghai Pudong Development Bank,58.796
4,Industrial Bank (China),57.388


Log the data as <code>"Extract phase Ended"</code>


In [ ]:
# Write your code here
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [ ]:
# Write your code here
log("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [43]:
# Call the function here
df = transform(df_bank_market)
# Print the first 5 rows here
df.head()

,Name,Market Cap (GBP$ Billion)
0,Banco Bilbao Vizcaya Argentaria,44.440
1,Japan Post Bank,44.328
2,The Bank of New York Mellon,43.814
3,Shanghai Pudong Development Bank,43.062
4,Industrial Bank (China),42.031


Log your data <code>"Transform phase Ended"</code>


In [45]:
# Write your code here
log("Transform phase Ended")

2021-12-26 19:29:01: Transform phase Ended


### Load


Log the following `"Load phase Started"`.


In [46]:
# Write your code here
log("Load phase Started")

2021-12-26 19:29:06: Load phase Started


Call the load function


In [50]:
# Write your code here
load(df_bank_market)

Log the following `"Load phase Ended"`.


In [51]:
# Write your code here
log("Load phase Ended")

2021-12-26 19:30:41: Load phase Ended


## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
